<a href="https://colab.research.google.com/github/ADecametre/H24-204-GR1-HandSpeak/blob/Entra%C3%AEnementMod%C3%A8le/Entrainement_du_modele_HandSpeak.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<link rel="stylesheet" href="/mediapipe/site.css">

# Personnalisation du modèle HandSpeak pour la reconnaissance de la langue des signes américaine.
Source : [Hand gesture recognition model customization guide  |  MediaPipe  |  Google for Developers](https://developers.google.com/mediapipe/solutions/customization/gesture_recognizer)
<table align="left" class="buttons">
  <td>
    <a href="https://github.com/googlesamples/mediapipe/blob/main/examples/customization/gesture_recognizer.ipynb" target="_blank">
      <img src="https://developers.google.com/static/mediapipe/solutions/customization/github-logo-32px_1920.png" alt="GitHub logo">
      Voir le code original sur GitHub
    </a>
  </td>
</table>

In [ ]:
#@title License information
# Copyright 2023 The MediaPipe Authors.
# Licensed under the Apache License, Version 2.0 (the "License");
#
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

Le package MediaPipe Model Maker est une solution à faible code pour personnaliser les modèles d'apprentissage automatique (ML).


Ce notebook nous permet de:
1.   Créer des données grâce à la webcam.
2.   Entrainer un modèle avec ces données.
3.   Tester et améliorer la performance du modèle.
4.   Télécharger le modèle.



## Préalables

Installer le package MediaPipe Model Maker.

In [ ]:
!pip install --upgrade pip
!pip install 'keras<3.0.0' mediapipe-model-maker
!pip install simplejson

Importer les librairies nécessaires.

In [ ]:
from google.colab import files
import os
import tensorflow as tf
assert tf.__version__.startswith('2')

from mediapipe_model_maker import gesture_recognizer

import matplotlib.pyplot as plt

## Création du modèle

Les étapes suivantes sont celles qu'on a entrepris pour créer notre modèle HandSpeak qui détecte le langage des signes.

### Capture et structuration des données

Le format requis pour l'ensemble de données de reconnaissance de gestes dans Model Maker est le suivant : <chemin_de_ensemble_de_données>/<nom_d'étiquette>/<nom_image>.*. De plus, l'un des noms d'étiquettes (label_names) doit être none. L'étiquette none représente tout geste qui n'est pas classifié comme l'un des autres gestes.

In [ ]:
dataset_path = "data" # @param {type:"string"}

In [ ]:
# @title Capture d'images par webcam
# @markdown <small>Si la webcam ne n'affiche pas, redémarrer le code.</small>
# @markdown <ul>
# @markdown <li>Le code utilise du JavaScript pour interagir avec la webcam et capturer des images. Il crée une interface utilisateur avec des boutons pour démarrer la capture, arrêter la capture et afficher la caméra en temps réel.</li>
# @markdown <li>Une fois que la capture est lancée, l'utilisateur peut entrer le nom de l'étiquette pour chaque signe à capturer.</li>
# @markdown <ul><li>Le code gère la création de nouveaux dossiers pour chaque étiquette si nécessaire, ainsi que la suppression du contenu des dossiers existants si l'utilisateur le souhaite.</li></ul></li>
# @markdown <li>Ensuite, l'utilisateur peut capturer des photos en cliquant sur le bouton "Capture". Les images capturées sont enregistrées dans des dossiers correspondant aux étiquettes.</li>
# @markdown <li>Lorsque l'utilisateur clique sur le bouton "Stop", le code demande l'étiquette du signe suivant pour lequelle on veut capturer des phtotos.</li>
# @markdown <li>Si l'utilisateur décide de ne pas ajouter une étiquette et clique sur le bouton "Annuler", alors le code s'arrête.</li>
# @markdown </ul>

from IPython.display import display, Javascript, JSON
from google.colab.output import eval_js
from base64 import b64decode
import time
import simplejson as json
import errno
import os
import shutil

def capture(quality=0.8):

  # CODE JAVASCRIPT

  js = Javascript('''

    // Initialise, puis affiche la webcam en tant réel
    async function initialiserWebcam(){
      // Création du HTML (div et boutons)
      document.body.innerHTML = "";
      const div = document.createElement('div');
      div.setAttribute("id", "divCapture");
      const capture = document.createElement('button');
      capture.setAttribute("id", "boutonCapture");
      const stop = document.createElement('button');
      stop.setAttribute("id", "boutonArretCapture");
      capture.textContent = 'Capture';
      stop.textContent = 'Stop';
      div.appendChild(capture);
      div.appendChild(stop);
      document.body.appendChild(div);

      // Connexion à la webcam
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      // Affichage vidéo
      const video = document.createElement('video');
      video.setAttribute("id", "webcam");
      video.style.display = 'block';
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);
    }

    // Capture la webcam et retourne l'image
    async function prendrePhoto(quality){
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);
      return await new Promise(resolve => {

        // Si le bouton Capture est cliqué
        capture = document.getElementById("boutonCapture");
        capture.onclick = ()=>{
          // Dessine la webcam sur un canvas
          const canvas = document.createElement('canvas');
          video = document.getElementById("webcam");
          canvas.width = video.videoWidth;
          canvas.height = video.videoHeight;
          canvas.getContext('2d').drawImage(video, 0, 0);
          // Retourne le canvas sous forme d'image
          resolve(canvas.toDataURL('image/jpeg', quality))
        };

        // Si le bouton Stop est cliqué
        stop = document.getElementById("boutonArretCapture");
        stop.onclick = ()=>{
          // Retourne null
          resolve(null);
        };

      });
    }

    // Désactive et met fin à l'affichage de la webcam
    async function arreterCapture(){
      const stream = await navigator.mediaDevices.getUserMedia({video: true});
      stream.getVideoTracks().forEach(track => track.stop);
      div = document.getElementById("divCapture");
      div.remove();
    }

  ''').data

  # Demande le nom du signe qui sera capturé
  def demander_label():
    return eval_js('prompt("Veuillez entrer le nom du label (ex. : A).")')

  # Exécute prendrePhoto()
  def prendre_photo():
    return eval_js('prendrePhoto({})'.format(quality))

  # Confirme la suppression ou non d'un dossier
  def confirmation_suppression(path):
    js_confirm = f'confirm("Voulez-vous supprimer le contenu du dossier {path} ?")'
    js_confirm_2 = f'confirm("Êtes-vous sûr de vouloir supprimer le contenu du dossier {path} ? Cette action est irréversible.")'
    return eval_js(js_confirm) and eval_js(js_confirm_2)


  # Exécute le code JavaScript
  eval_js(js)
  eval_js('initialiserWebcam()')

  # Demande le nom du signe à capturer (tant qu'il y en a)
  label = demander_label()
  while(label != None):
    path = f'{dataset_path}/{label}'
    try:
      # Crée un dossier avec le nom du signe
      os.makedirs(path)
    except OSError as exc:
      if exc.errno == errno.EEXIST and os.path.isdir(path):
        # Si le dossier existe déjà, confirme la suppression ou non du dossier
        if(confirmation_suppression(path)):
          shutil.rmtree(path)
          os.makedirs(path)
        pass
      else: raise

    # Prend la photo (tant qu'il y en a) puis l'enregistre dans le dossier
    data = prendre_photo()
    while(data != None):
      binary = b64decode(data.split(',')[1])
      with open(f'{path}/{time.time()}.jpg', 'wb+') as f:
        f.write(binary)
      data = prendre_photo()

    label = demander_label()

  # Arrête la capture lorsqu'il n'y a plus de signe à capturer
  eval_js('arreterCapture()')
  return

try:
  capture()
except Exception as err:
  print(str(err))

Vérifier que l'on a tout les catégories de signe de main capturées avec la webcam.

In [ ]:
!rm -rf `find -type d -name .ipynb_checkpoints`
print(dataset_path)
labels = []
for i in os.listdir(dataset_path):
  if os.path.isdir(os.path.join(dataset_path, i)):
    labels.append(i)
print(labels)

### Entraînement de l'IA
L'entraînement de l'IA va se faire sur 4 étapes.

#### Charger le dataset

Chargez le dataset situé à `chemin_du_dataset` en utilisant la méthode `Dataset.from_folder`. Lors du chargement du dataset, on exécute le modèle de détection de main pré-emballé de MediaPipe Hands pour détecter les repères de la main à partir des images. Toutes les images sans mains détectées sont omises du dataset. Le dataset résultant contiendra les positions des repères de la main extraites de chaque image, plutôt que les images elles-mêmes.

La classe `HandDataPreprocessingParams` contient deux options configurables pour le processus de chargement des données :

* `shuffle` : Un booléen contrôlant s'il faut mélanger le jeu de données. Par défaut, il est défini sur vrai.
* `min_detection_confidence` : Un float compris entre 0 et 1 contrôlant le seuil de confiance pour la détection de main.

On divise le dataset : 80% pour l'entraînement, 10% pour la validation et 10% pour les tests.

In [ ]:
# @title Paramètres de chargement du dataset

# @markdown #### `HandDataPreprocessingParams`
shuffle = True # @param {type:"boolean"}
min_detection_confidence = 0.7 # @param {type:"slider", min:0, max:1, step:0.01}
# @markdown <sup><sub>Valeurs par défaut : True, 0.7 </sup></sub>

# @markdown #### `Division des données`
data_split = 0.8 # @param {type:"slider", min:0, max:1, step:0.01}
rest_data_split = 0.5 # @param {type:"slider", min:0, max:1, step:0.01}
# @markdown <sup><sub>Valeurs par défaut : 0.8, 0.5 </sup></sub>


data = gesture_recognizer.Dataset.from_folder(
    dirname=dataset_path,
    hparams=gesture_recognizer.HandDataPreprocessingParams(shuffle=shuffle, min_detection_confidence=min_detection_confidence)
)
train_data, rest_data = data.split(data_split)
validation_data, test_data = rest_data.split(rest_data_split)

#### Entraîner le modèle

 Entraîner le reconnaisseur de gestes personnalisé en utilisant la méthode create et en passant les données d'entraînement, les données de validation, les options du modèle et les hyperparamètres. Pour plus d'informations sur les options du modèle et les hyperparamètres, consultez la section [Hyperparamètres](#scrollTo=F1tiLGGRcvhy) ci-dessus.

In [ ]:
hparams = gesture_recognizer.HParams(export_dir="exported_model")
options = gesture_recognizer.GestureRecognizerOptions(hparams=hparams)
model = gesture_recognizer.GestureRecognizer.create(
    train_data=train_data,
    validation_data=validation_data,
    options=options
)

#### Évaluer la performance du modèle

Après avoir entraîné le modèle, on l'évalue sur un dataset de test et on imprime les métriques de perte et de précision.

In [ ]:
loss, acc = model.evaluate(test_data, batch_size=1)
print(f"Test loss:{loss}, Test accuracy:{acc}")

#### Exporter à Tensorflow Lite Model

Après avoir créé le modèle, on le convertit et on l'exporte au format de modèle Tensorflow Lite pour une utilisation ultérieure dans une application sur appareil. L'exportation comprend également les métadonnées du modèle, qui incluent le fichier d'étiquettes.

In [ ]:
model.export_model()
!ls exported_model

In [ ]:
files.download('exported_model/gesture_recognizer.task')

## Hyperparamètres

On peut personnaliser davantage le modèle en utilisant la classe `GestureRecognizerOptions`, qui comporte deux paramètres optionnels pour `ModelOptions` et `HParams`. On utilise la classe `ModelOptions` pour personnaliser les paramètres liés au modèle lui-même, et la classe `HParams` pour personnaliser les autres paramètres liés à l'entraînement et à la sauvegarde du modèle.

[`ModelOptions`](https://developers.google.com/mediapipe/api/solutions/python/mediapipe_model_maker/gesture_recognizer/ModelOptions) possède un paramètre personnalisable qui affecte la précision :
* `dropout_rate` : La fraction des unités d'entrée à supprimer. Utilisé dans la couche de désactivation aléatoire (dropout). Par défaut, 0,05.
* `layer_widths` : Une liste des largeurs de couche cachée pour le modèle de gestes. Chaque élément de la liste créera une nouvelle couche cachée avec la largeur spécifiée. Les couches cachées sont séparées avec BatchNorm, Dropout et ReLU. Par défaut, une liste vide (pas de couches cachées).

[`HParams`](https://developers.google.com/mediapipe/api/solutions/python/mediapipe_model_maker/gesture_recognizer/HParams) possède la liste suivante de paramètres personnalisables qui affectent la précision du modèle :
* `learning_rate` : Le taux d'apprentissage à utiliser pour l'entraînement par descente de gradient. Par défaut, 0,001.
* `batch_size` : Taille du lot pour l'entraînement. Par défaut, 2.
* `epochs` : Nombre d'itérations d'entraînement sur l'ensemble de données. Par défaut, 10.
* `steps_per_epoch` : Un entier facultatif qui indique le nombre de pas d'entraînement par époque. Si non défini, le pipeline d'entraînement calcule le nombre de pas par époque par défaut comme la taille de l'ensemble de données d'entraînement divisée par la taille du lot.
* `shuffle` : True si l'ensemble de données est mélangé avant l'entraînement. Par défaut, False.
* `lr_decay` : Taux de décroissance du taux d'apprentissage à utiliser pour l'entraînement par descente de gradient. Par défaut, 0,99.
* `gamma` : Paramètre gamma pour la perte focal. Par défaut, 2.

Paramètre supplémentaire de `HParams` qui n'affecte pas la précision du modèle :
* `export_dir` : L'emplacement des fichiers de point de contrôle du modèle et des fichiers de modèle exportés.

Par exemple, on peut entraîner un nouveau modèle avec un taux de désactivation (dropout_rate) de 0,2 et un taux d'apprentissage (learning_rate) de 0,003.

In [ ]:
# @title Hyperparamètres

# @markdown #### `ModelOptions`
dropout_rate = 0.2 # @param {type:"number", placeholder:"0.003"}
layer_widths = [] # @param {type:"raw"}

# @markdown #### `HParams`
learning_rate = 0.003 # @param {type:"number"}
batch_size = 2 # @param {type:"integer"}
epochs = 10 # @param {type:"integer"}
steps_per_epoch = None # @param {type:"raw"}
shuffle = False # @param {type:"boolean"}
export_dir = "exported_model_2" # @param {type:"string"}
lr_decay = 0.99 # @param {type:"number"}
gamma = 2 # @param {type:"integer"}

hparams = gesture_recognizer.HParams(learning_rate=learning_rate, batch_size=batch_size, epochs=epochs, steps_per_epoch=steps_per_epoch, shuffle=shuffle, export_dir=export_dir, lr_decay=lr_decay, gamma=gamma)
model_options = gesture_recognizer.ModelOptions(dropout_rate=dropout_rate, layer_widths=layer_widths)
options = gesture_recognizer.GestureRecognizerOptions(model_options=model_options, hparams=hparams)
model_2 = gesture_recognizer.GestureRecognizer.create(
    train_data=train_data,
    validation_data=validation_data,
    options=options
)

Evaluer le nouveau modèle entrainé.

In [ ]:
loss, accuracy = model_2.evaluate(test_data)
print(f"Test loss:{loss}, Test accuracy:{accuracy}")

Exporter le nouveau modèle.

In [ ]:
model_2.export_model()
!ls $export_dir

In [ ]:
files.download(f'{export_dir}/gesture_recognizer.task')